<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')
 
#train = pd.read_csv('../input/dsn-ai-bootcamp-data/Train.csv')
#test = pd.read_csv('../input/dsn-ai-bootcamp-data/Test.csv')
#sub = pd.read_csv('../input/dsn-ai-bootcamp-data/SampleSubmission.csv')

In [4]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [7]:
#pip install catboost

In [8]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [9]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [ ]:
# features

In [10]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [31]:
train['form_field1*formfield2'] = train['form_field1'] * train['form_field2']
train['form_field1/formfield2'] = train['form_field1'] / train['form_field2']
 
train['form_field1*formfield3'] = train['form_field1'] * train.form_field3
train['form_field1*formfield4'] = train.form_field1 * train.form_field4
train['form_field1*formfield5'] = train.form_field1 * train.form_field5
 
#train['form_field30*formfield41'] = train['form_field30'] * train['form_field41']
#train['form_field31*formfield40'] = train['form_field31'] * train['form_field40']
 
train['form_field6+field7'] =  (train.form_field6 + train.form_field8)/2
 
train['form_field1*formfield6'] = train['form_field1'] * train['form_field6']
train['form_field2*formfield6'] = train['form_field2'] * train['form_field6']
 
train['form_field1*formfield9'] = train['form_field1'] * train['form_field9']
train['form_field2*formfield9'] = train['form_field2'] * train['form_field9']
 
train['form_field1*formfield37'] = train['form_field1'] * train['form_field37']
train['form_field2*formfield37'] = train['form_field2'] * train['form_field37']
 
train['form_field9*formfield6'] = train['form_field9'] * train['form_field6']
train['form_field37*formfield6'] = train['form_field37'] * train['form_field6']
 
train['form_field9*formfield37'] = train['form_field9'] * train['form_field37']
train['form_field1*formfield8'] = train['form_field1'] * train['form_field8']
 
 
 
 
#train['form_field13/field14'] =(train.form_field13 / train.form_field14)*100
 
#train['form_field32/field33'] = (train['form_field32']+ train['form_field33'])/2
#train['form_field36/field38'] = train['form_field36'] / train['form_field38']
#train['form_field36/field39'] = train['form_field36'] / train['form_field39']
#train['form_field36/field18'] = train['form_field36'] / train['form_field18']
#train['form_field36/field20'] = train['form_field36'] / train['form_field20']
#train['form_field44*field50'] = train['form_field44'] * train['form_field50']

In [33]:
test['form_field1*formfield2'] = test['form_field1'] * test['form_field2']
test['form_field1/formfield2'] = test['form_field1'] / test['form_field2']
 
test['form_field1*formfield3'] = test['form_field1'] * test.form_field3
test['form_field1*formfield4'] = test.form_field1 * test.form_field4
test['form_field1*formfield5'] = test.form_field1 * test.form_field5
 
#test['form_field30*formfield41'] = test['form_field30'] * test['form_field41']
#test['form_field31*formfield40'] = test['form_field31'] * test['form_field40']
 
test['form_field6+field7'] =  (test.form_field6 + test.form_field8)/2
 
test['form_field1*formfield6'] = test['form_field1'] * test['form_field6']
test['form_field2*formfield6'] = test['form_field2'] * test['form_field6']
 
test['form_field1*formfield9'] = test['form_field1'] * test['form_field9']
test['form_field2*formfield9'] = test['form_field2'] * test['form_field9']
 
test['form_field1*formfield37'] = test['form_field1'] * test['form_field37']
test['form_field2*formfield37'] = test['form_field2'] * test['form_field37']
 
test['form_field9*formfield6'] = test['form_field9'] * test['form_field6']
test['form_field37*formfield6'] = test['form_field37'] *test['form_field6']
 
test['form_field9*formfield37'] = test['form_field9'] * test['form_field37']
test['form_field1*formfield8'] = test['form_field1'] * test['form_field8']
 
 
 
#test['form_field13/field14'] = (test.form_field13 / test.form_field14)*100
#test['form_field32/field33'] = (test['form_field32']+ test['form_field33'])/2
#test['form_field36/field38'] = test['form_field36'] / test['form_field38']
#test['form_field36/field39'] = test['form_field36'] / test['form_field39']
#test['form_field36/field18'] = test['form_field36'] / test['form_field18']
 
#test['form_field36/field20'] = test['form_field36'] / test['form_field20']
#test['form_field44*field50'] =test['form_field44'] * test['form_field50']

In [ ]:
 for col in features: 
   
 
    new_train[col + '_mean_on_field1_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field1']]).transform('mean')
    new_train[col + '_mean_on_field2_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field2']]).transform('mean')
 
    new_train[col + '_sum_on_field1_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field1']]).transform('sum')
    new_train[col + '_sum_on_field2_'] = new_train[col].groupby([new_train['form_field47'],new_train['form_field2']]).transform('sum')

In [34]:
X = train[features]
y = train['default_status']

In [35]:
from sklearn.model_selection import StratifiedKFold

In [36]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [37]:
# Specify number of folds
n_skf = 15
kf = StratifiedKFold(n_skf)
 
#seed = 42
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {'max_depth':10,
    'n_estimators': 5000,
    'learning_rate': 0.008,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [29]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU',random_seed = i)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=0)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

Fold 0 : 0.8298943352775953
Fold 1 : 0.8511385465915325
Fold 2 : 0.8406387569129851
Fold 3 : 0.8471893493991784
Fold 4 : 0.8239886639644721
Fold 5 : 0.8450740248842841
Fold 6 : 0.8432417411391753
Fold 7 : 0.8416679409725968
Fold 8 : 0.8418903299973686
Fold 9 : 0.8246068604491406
Fold 10 : 0.8205398192788386
Fold 11 : 0.850269311944658
Fold 12 : 0.8674134487531078
Fold 13 : 0.8368557219182431
Fold 14 : 0.8489338134555839


Avg log :  0.8408895109959174


In [30]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'15 fold CV, score: 0.8408895109959174'

'15 fold CV, score: 0.8416762360251805', LR-0.005, n_estimator = 10000, seed = i, depth = 7, submit10.csv

'15 fold CV, score: 0.8416215384358319', LR-0.008, n_estimator = 5000, seed = i, depth = 8

'20 fold CV, score: 0.8419991965085709',LR-0.008, n_estimator = 5000, seed = i, depth = 7, submit9.csv

'15 fold CV, score: 0.8419018702724905', LR-0.008, n_estimator = 5000, seed = i, depth = 7, submit8.csv, Best score on LB

'15 fold CV, score: 0.8418325697189986, LR-0.008, n_estimator = 5000, seed = 42, depth = 7, submit7.csv

'10 fold CV, score: 0.8416283637107365', LR-0.008, n_estimator = 5000, seed = 42, depth = 7, submit5.csv

'10 fold CV, score: 0.8404761359972677', LR-0.008, n_estimator = 5000, seed = 42, depth = 10, submit4.csv

'10 fold CV, score: 0.8416127641119365', LR-0.008, n_estimator = 5000, seed = 2020

'10 fold CV, score: 0.8415039416559706', LR-0.008, n_estimator = 5000, seed = 42

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [21]:
oof_prediction = pd.DataFrame(test_oofs).T

In [22]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [23]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12,fold_13,fold_14,fold_15
0,0.314115,0.292056,0.342378,0.320132,0.310555,0.307088,0.314581,0.349372,0.331286,0.302720,0.328746,0.316901,0.293201,0.322182,0.304439
1,0.425213,0.341592,0.377943,0.394663,0.331499,0.398646,0.396911,0.444832,0.384766,0.376909,0.384882,0.352310,0.384718,0.397946,0.351174
2,0.354195,0.389987,0.381181,0.368937,0.395171,0.418407,0.415784,0.374056,0.396557,0.397112,0.410189,0.386720,0.358726,0.359079,0.369657
3,0.729698,0.739634,0.763501,0.741557,0.752966,0.764886,0.742584,0.752692,0.729274,0.761641,0.759470,0.751393,0.740636,0.725104,0.738868
4,0.167424,0.162916,0.173079,0.177167,0.158162,0.158191,0.158758,0.174698,0.185564,0.166953,0.154338,0.177701,0.187672,0.174128,0.151464


In [24]:
sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [ ]:
### We take the mean of all 5 predictions to generate submission

In [25]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [26]:
sub.to_csv('submit11.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.